# Download data from SABIO-RK

In [17]:
%run ../bigg_sabio/scraper.py
bgsb = SABIO_scraping('../bigg_models/Ecoli core, BiGG, indented.json', 'test_Ecoli-3')
# %time bgsb.scrape_bigg_xls()
# %time  bgsb._glob_csv()
%time bgsb._scrape_entryids()

C:\Program Files (x86)\Anaconda3\lib\site-packages\IPython\core\magic.py:187: DtypeWarning: Columns (19) have mixed types.Specify dtype option on import or set low_memory=False.
  call = lambda f, *a, **k: f(*a, **k)


Continuing Step 1
Estimated completion of scraping the XLS data for test_Ecoli-3: 2022-02-09 15:55:40.210190, in 91.025 hours
Scraped entryID 10922/10923	2022-02-06 07:43:25.335120The parameter specifications for each entryid have been scraped.
Wall time: 10h 49min 28s


# Refine data into FBA input files

In [83]:
%run ../bigg_sabio/scraper.py
bgsb = SABIO_scraping('../bigg_models/Ecoli core, BiGG, indented.json', 'test_Ecoli-3', verbose = True)
%time bgsb.to_fba()

C:\Program Files (x86)\Anaconda3\lib\site-packages\IPython\core\magic.py:187: DtypeWarning: Columns (19) have mixed types.Specify dtype option on import or set low_memory=False.
  call = lambda f, *a, **k: f(*a, **k)




SABIO reaction: 2-Oxoglutarate = 2-Oxoglutarate
['akg']
The || 2-Oxoglutarate <-> 2-Oxoglutarate || reaction with ['2-Oxoglutarate', '2-Oxoglutarate'] chemicals does not match the BiGG reaction of ['akg'] chemicals.


SABIO reaction: 2-Oxoglutarate + NAD+ + Coenzyme A = H+ + Succinyl-CoA + NADH + CO2
-->ERROR: The NAD+ metabolite in is not recognized by BiGG.
The 11896 entryid has an unexpected rate_law unknown.
The 11903 entryid has an unexpected rate_law unknown.
The 11904 entryid has an unexpected rate_law unknown.
The 11905 entryid has an unexpected rate_law unknown.
The 11906 entryid has an unexpected rate_law unknown.
The 11907 entryid has an unexpected rate_law unknown.
The 11908 entryid has an unexpected rate_law unknown.
The 11909 entryid has an unexpected rate_law unknown.
The 11910 entryid has an unexpected rate_law unknown.
The 11911 entryid has an unexpected rate_law unknown.
The 11912 entryid has an unexpected rate_law unknown.
The 11913 entryid has an unexpected rate_l

C:\Users\Andrew Freiburger\Dropbox\My PC (DESKTOP-M302P50)\Documents\UVic Civil Engineering\dFBA\BiGG_SABIO\bigg_sabio\scraper.py:1011: UserWarning: ---> ERROR: The rate rate Vmax*A/(Km+A) does not reflect {'Vmax': {'type': 'Vmax', 'species': '-', 'start val.': '0.3', 'end val.': '-', 'deviat.': '-', 'unit': 'U/mg', 'comment': '-'}, 'Km': {'type': 'Km', 'species': '2-Oxoglutarate', 'start val.': '2500.0', 'end val.': '-', 'deviat.': '-', 'unit': 'µM', 'comment': 'apparent'}} in the defined entry variables and parameters.
  warnings.warn(f'---> ERROR: The rate rate {rate_law} does not reflect {substituted_parameters} in the defined entry variables and parameters.')
C:\Users\Andrew Freiburger\Dropbox\My PC (DESKTOP-M302P50)\Documents\UVic Civil Engineering\dFBA\BiGG_SABIO\bigg_sabio\scraper.py:1011: UserWarning: ---> ERROR: The rate rate (Vmax*A^n)/(S_half^n+A^n) does not reflect {'n': {'type': 'Hill coefficient', 'species': '2-Oxoglutarate', 'start val.': '0.92', 'end val.': '-', 'devia



SABIO reaction: Coenzyme A + NAD+ + alpha-Ketovaline = NADH + CO2 + 2-Methylpropanoyl-CoA + H+
-->ERROR: The NAD+ metabolite in is not recognized by BiGG.
-->ERROR: The alpha-Ketovaline metabolite in is not recognized by BiGG.
-->ERROR: The 2-Methylpropanoyl-CoA metabolite in is not recognized by BiGG.
The 44329 entryid has an unexpected rate_law -.
The 44330 entryid has an unexpected rate_law -.


SABIO reaction: Coenzyme A + NAD+ + 4-Methyl-2-oxopentanoate = CO2 + NADH + 3-Methylbutanoyl-CoA + H+
-->ERROR: The NAD+ metabolite in is not recognized by BiGG.
-->ERROR: The 3-Methylbutanoyl-CoA metabolite in is not recognized by BiGG.


SABIO reaction: NAD+ + Coenzyme A + 3-Methyl-2-oxopentanoate = NADH + CO2 + 2-Methylbutanoyl-CoA + H+
-->ERROR: The NAD+ metabolite in is not recognized by BiGG.
-->ERROR: The 3-Methyl-2-oxopentanoate metabolite in is not recognized by BiGG.


SABIO reaction: NAD+ + Dihydrolipoylprotein = NADH + H+ + Lipoylprotein
-->ERROR: The NAD+ metabolite in is not 

C:\Users\Andrew Freiburger\Dropbox\My PC (DESKTOP-M302P50)\Documents\UVic Civil Engineering\dFBA\BiGG_SABIO\bigg_sabio\scraper.py:1011: UserWarning: ---> ERROR: The rate rate kcat*E*S/(Km+S) does not reflect {'kcat': {'type': 'kcat', 'species': '-', 'start val.': '17.2', 'end val.': '-', 'deviat.': '0.5', 'unit': 's^(-1)', 'comment': '-'}, 'Km': {'type': 'Km', 'species': 'Acetaldehyde', 'start val.': '23.6', 'end val.': '-', 'deviat.': '1.8', 'unit': 'mM', 'comment': 'apparent'}} in the defined entry variables and parameters.
  warnings.warn(f'---> ERROR: The rate rate {rate_law} does not reflect {substituted_parameters} in the defined entry variables and parameters.')
C:\Users\Andrew Freiburger\Dropbox\My PC (DESKTOP-M302P50)\Documents\UVic Civil Engineering\dFBA\BiGG_SABIO\bigg_sabio\scraper.py:1011: UserWarning: ---> ERROR: The rate rate kcat*E*S3/(Km+S3) does not reflect {'kcat': {'type': 'kcat', 'species': '-', 'start val.': '14.1', 'end val.': '-', 'deviat.': '0.2', 'unit': 's^(-

...


The 48655 entryid is missing from the datafile.
The 48656 entryid is missing from the datafile.
The 48657 entryid is missing from the datafile.
The 48658 entryid is missing from the datafile.
The 48659 entryid is missing from the datafile.
The 48660 entryid is missing from the datafile.
The 48661 entryid has an unexpected rate_law -.
The 48662 entryid has an unexpected rate_law -.
The 48663 entryid has an unexpected rate_law -.
The 48664 entryid has an unexpected rate_law -.
The 48665 entryid is missing from the datafile.
The 48666 entryid is missing from the datafile.
The 52806 entryid has an unexpected rate_law -.
The 52807 entryid has an unexpected rate_law -.
The 52808 entryid has an unexpected rate_law -.
The 52809 entryid has an unexpected rate_law -.
The 52810 entryid has an unexpected rate_law -.
The 52811 entryid has an unexpected rate_law -.
The 52812 entryid has an unexpected rate_law -.
The 52813 entryid has an unexpected rate_law -.
The 52814 entryid is missing from the d

C:\Users\Andrew Freiburger\Dropbox\My PC (DESKTOP-M302P50)\Documents\UVic Civil Engineering\dFBA\BiGG_SABIO\bigg_sabio\scraper.py:1011: UserWarning: ---> ERROR: The rate rate (Vmax*S^n)/(S_half^n+S^n) does not reflect {'n': {'type': 'Hill coefficient', 'species': 'Phosphoenolpyruvate', 'start val.': '1.6', 'end val.': '-', 'deviat.': '0.1', 'unit': '-', 'comment': '-'}, 'Vmax': {'type': 'Vmax', 'species': '-', 'start val.': '310.0', 'end val.': '-', 'deviat.': '37', 'unit': 'U/mg', 'comment': '-'}, 'S_half': {'type': 'Km', 'species': 'Phosphoenolpyruvate', 'start val.': '2.1', 'end val.': '-', 'deviat.': '0.7', 'unit': 'mM', 'comment': '-'}} in the defined entry variables and parameters.
  warnings.warn(f'---> ERROR: The rate rate {rate_law} does not reflect {substituted_parameters} in the defined entry variables and parameters.')
C:\Users\Andrew Freiburger\Dropbox\My PC (DESKTOP-M302P50)\Documents\UVic Civil Engineering\dFBA\BiGG_SABIO\bigg_sabio\scraper.py:1011: UserWarning: ---> ERR



SABIO reaction: D-Ribose 5-phosphate = D-Ribulose 5-phosphate
-->ERROR: The D-Ribose 5-phosphate metabolite in is not recognized by BiGG.
The 24911 entryid has an unexpected rate_law unknown.
The 24912 entryid has an unexpected rate_law unknown.
The 24913 entryid has an unexpected rate_law unknown.
The 54895 entryid is missing from the datafile.
The 54896 entryid has an unexpected rate_law -.
The 54897 entryid has an unexpected rate_law -.
The 54898 entryid has an unexpected rate_law -.
The 54899 entryid has an unexpected rate_law -.
The 54900 entryid has an unexpected rate_law -.
The kcat parameter has an unaccepted value None.
The 55473 entryid has an unexpected rate_law -.
The 55474 entryid has an unexpected rate_law -.
The 55475 entryid has an unexpected rate_law -.
The 55476 entryid has an unexpected rate_law -.
The 55477 entryid has an unexpected rate_law -.
The 55478 entryid has an unexpected rate_law -.
The kcat parameter has an unaccepted value None.
The kcat parameter has a

C:\Users\Andrew Freiburger\Dropbox\My PC (DESKTOP-M302P50)\Documents\UVic Civil Engineering\dFBA\BiGG_SABIO\bigg_sabio\scraper.py:1011: UserWarning: ---> ERROR: The rate rate Vmax*A/(Km+A) does not reflect {'Vmax': {'type': 'Vmax', 'species': '-', 'start val.': '6.61', 'end val.': '-', 'deviat.': '-', 'unit': 'U/mg', 'comment': 'purified enzyme'}, 'Km': {'type': 'Km', 'species': 'Succinate', 'start val.': '5.1', 'end val.': '-', 'deviat.': '1.7', 'unit': 'mM', 'comment': 'apparent'}} in the defined entry variables and parameters.
  warnings.warn(f'---> ERROR: The rate rate {rate_law} does not reflect {substituted_parameters} in the defined entry variables and parameters.')
C:\Users\Andrew Freiburger\Dropbox\My PC (DESKTOP-M302P50)\Documents\UVic Civil Engineering\dFBA\BiGG_SABIO\bigg_sabio\scraper.py:1011: UserWarning: ---> ERROR: The rate rate Vmax*A/(Km+A) does not reflect {'Vmax': {'type': 'Vmax', 'species': '-', 'start val.': '6.61', 'end val.': '-', 'deviat.': '-', 'unit': 'U/mg',


The 28537 entryid has an unexpected rate_law unknown.
The 28627 entryid has an unexpected rate_law unknown.
The 28628 entryid has an unexpected rate_law unknown.
The 28629 entryid has an unexpected rate_law unknown.
The 28630 entryid has an unexpected rate_law unknown.
The 28631 entryid has an unexpected rate_law unknown.
The 28632 entryid has an unexpected rate_law unknown.
The 28633 entryid has an unexpected rate_law unknown.
The 28634 entryid has an unexpected rate_law unknown.
The 28635 entryid has an unexpected rate_law unknown.
The 28636 entryid has an unexpected rate_law unknown.
The 28637 entryid has an unexpected rate_law unknown.
The 28638 entryid has an unexpected rate_law unknown.
The 28639 entryid has an unexpected rate_law unknown.
The 28640 entryid has an unexpected rate_law unknown.
The 28641 entryid has an unexpected rate_law unknown.
The 28642 entryid has an unexpected rate_law unknown.
The 28643 entryid has an unexpected rate_law unknown.
The 28644 entryid has an un